In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from sentence_transformers import SentenceTransformer
from torch.utils.data import DataLoader, Dataset
from pytorch_lightning.callbacks import Callback
import pandas as pd
import random
# train_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/traindata_movielens.csv')

# Select 100 random samples
# train_ratings = train_ratings.sample(n=10, random_state=42)
ratings = torch.tensor([float(random.randint(1, 5)) for _ in range(5)])


D:\Anaconda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
titles = ["Inception", "Titanic", "Avatar", "The Matrix", "The Godfather", "Star Wars", "The Dark Knight", "Pulp Fiction", "Forrest Gump", "Jurassic Park"]
genres = ["Action", "Drama", "Sci-Fi", "Romance", "Thriller", "Comedy", "Adventure", "Fantasy", "Horror", "Mystery"]
years = ["1994", "1999", "2008", "2010", "2014", "2018", "2020", "1984", "1995", "2001"]

movies = []
for _ in range(5):
    movie = random.choice(titles)
    genre = random.choice(genres)
    year = random.choice(years)
    sample = f"title: {movie} [SEP] genre: {genre} [SEP] year: {year}"
    movies.append(sample)

In [ ]:
import random

occupations = ["doctor", "engineer", "teacher", "nurse", "writer", "musician", "artist", "lawyer", "scientist", "pilot"]
ages = ["18-24", "25-34", "35-44", "45-54", "55-64", "65-74", "75+"]
genders = ["Male", "Female"]

# Generate 10 random samples
users = []
for _ in range(5):
    occupation = random.choice(occupations)
    age = random.choice(ages)
    gender = random.choice(genders)
    sample = f"profession: {occupation} [SEP] age range: {age} [SEP] gender: {gender}"
    users.append(sample)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# torch.cuda.get_device_name(0)

In [9]:
users = [
    "profession: teacher [SEP] age range: 65-74",
    "profession: musician [SEP] age range: 75+",
    "profession: teacher [SEP] age range: 25-55",
    "profession: musician [SEP] age range: 65-74",
    "profession: teacher [SEP] age range: 45-55",
]

In [3]:
users = pd.read_csv('processed_dataset/MovieLens-1M/users/users_movielens.csv')
movies = pd.read_csv('processed_dataset/MovieLens-1M/movies/movies_movielens.csv')
train_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/traindata_movielens.csv')
val_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/valdata_movielens.csv')
test_ratings = pd.read_csv('processed_dataset/MovieLens-1M/ratings/testdata_movielens.csv')

In [4]:
# Combine user features into a single string for each user
users['user_features'] = 'occupation: ' + users['occupation'] + ' [SEP] age: ' + users['age'].astype(str) + ' [SEP] gender: ' + users['gender'].astype(str)

# Combine movie features into a single string for each movie
movies['movie_features'] = 'title: ' + movies['title'] + ' [SEP] genres: ' + movies['genres']

In [5]:
# Create a dictionary for fast lookup
user_features_dict = users.set_index('user_id')['user_features'].to_dict()
movie_features_dict = movies.set_index('item_id')['movie_features'].to_dict()

# Create lists of user and item texts
train_user_texts = [user_features_dict[userId] for userId in train_ratings['user_id']]
train_item_texts = [movie_features_dict[movieId] for movieId in train_ratings['item_id'].unique()]

# Create lists of user and item texts
val_user_texts = [user_features_dict[userId] for userId in val_ratings['user_id']]
val_item_texts = [movie_features_dict[movieId] for movieId in val_ratings['item_id'].unique()]

train_item_id_to_idx = {movieId: idx for idx, movieId in enumerate(train_ratings['item_id'].unique())}
val_item_id_to_idx = {movieId: idx for idx, movieId in enumerate(val_ratings['item_id'].unique())}
test_item_id_to_idx = {movieId: idx for idx, movieId in enumerate(test_ratings['item_id'].unique())}

test_item_texts = [movie_features_dict[movieId] for movieId in test_ratings['item_id'].unique()]

In [7]:
train_item_texts[0]

'title: Silence of the Lambs, The (1991) [SEP] genres: Drama|Thriller'

In [408]:
movie_features_dict[1434]

'title: Stranger, The (1994) [SEP] genres: Action'

In [45]:
# train_item_texts[item_id_to_idx.get(train_ratings['item_id'][0])]
item_id = 858
text = val_item_texts[val_item_id_to_idx[item_id]] if item_id in val_item_id_to_idx else "Item ID not found"
text

'title: Godfather, The (1972) [SEP] genres: Action|Crime|Drama'

In [57]:
# Create a movies dataset
movies_data = {
    'movie_id': [1, 2, 3, 4, 5],
    'title': ['Movie A', 'Movie B', 'Movie C', 'Movie D', 'Movie E'],
    'genre': ['Action', 'Comedy', 'Drama', 'Horror', 'Sci-Fi']
}
movies_df = pd.DataFrame(movies_data)

# Create a users dataset
users_data = {
    'user_id': [101, 102, 103, 104, 105],
    'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva'],
    'age': [25, 30, 35, 40, 45]
}
users_df = pd.DataFrame(users_data)
from sklearn.model_selection import train_test_split

# Create a ratings dataset
ratings_data = {
    'user_id': [101, 102, 103, 104, 105, 106, 107, 108, 109, 110],
    'movie_id': [1, 2, 3, 4, 5, 1, 2, 3, 4, 5],
    'rating': [4.5, 3.0, 5.0, 2.5, 4.0, 3.5, 4.0, 2.0, 5.0, 3.5]
}
ratings_df = pd.DataFrame(ratings_data)

# Split the dataset into train, validation, and test sets
train_df, temp_df = train_test_split(ratings_df, test_size=0.4, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


In [59]:
val_df

,user_id,movie_id,rating
8,109,4,5.0
5,106,1,3.5


In [60]:

class UserItemDataset(Dataset):
    def __init__(self, users, items, ratings):
        self.users = users
        self.items = items
        self.ratings = ratings
    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return {
            'user': self.users[idx],
            'item': self.items[idx],
            'rating': self.ratings[idx]
        }

In [7]:
data_length = len(train_user_texts)
random_indices = random.sample(range(data_length), 10)

random_user_texts = [train_user_texts[i] for i in random_indices]
random_item_ids = [train_ratings['item_id'][i] for i in random_indices]
random_ratings = [train_ratings['rating'][i] for i in random_indices]

In [8]:
random_indices = random.sample(range(data_length), 10)

val_random_user_texts = [train_user_texts[i] for i in random_indices]
val_random_item_ids = [train_ratings['item_id'][i] for i in random_indices]
val_random_ratings = [train_ratings['rating'][i] for i in random_indices]

In [80]:
random_item_ids

[3052, 233, 3208, 52, 2858, 2985, 3755, 653, 921, 2291]

In [68]:
# dataset = UserItemDataset(users, movies, ratings)
# train_dataset = UserItemDataset(random_user_texts, random_item_ids, random_ratings)
# val_dataset = UserItemDataset(val_random_user_texts, val_random_item_ids, val_random_ratings)

train_dataset = UserItemDataset(train_user_texts[:10], train_ratings['item_id'], train_ratings['rating'][:10])
val_dataset = UserItemDataset(val_user_texts[:5], val_ratings['item_id'], val_ratings['rating'][:5])

# dataset[0]

In [69]:
users_df

,user_id,name,age
0,101,Alice,25
1,102,Bob,30
2,103,Charlie,35
3,104,David,40
4,105,Eva,45


In [70]:
train_dataset.users[0]

'Alice'

In [71]:
train_dataset.items[0]

'Movie A'

In [72]:
train_dataset.ratings[0]

KeyError: 0

In [10]:

class TwoTowerModel(pl.LightningModule):
    def __init__(self, user_model_name, item_model_name, embedding_size=384):
        super(TwoTowerModel, self).__init__()
        self.user_model = SentenceTransformer(user_model_name)
        self.item_model = SentenceTransformer(item_model_name)

        self.user_fc = nn.Linear(embedding_size, embedding_size)
        self.item_fc = nn.Linear(embedding_size, embedding_size)

        self.criterion = nn.MSELoss()
        self.epoch_losses = {'train_loss': [], 'val_loss': []}

    def forward(self, user_text, item_text, step):
        if step == 'train':
            item_text = [train_item_texts[train_item_id_to_idx.get(item_text)] for item_text in item_text.tolist()]
        elif step == 'val':
            item_text = [val_item_texts[val_item_id_to_idx.get(item_text)] for item_text in item_text.tolist()]
        else:
            item_text = [test_item_texts[test_item_id_to_idx.get(item_text)] for item_text in item_text.tolist()]

        # print(user_text, item_text)
        user_embedding = self.user_model.encode(user_text, convert_to_tensor=True)
        item_embedding = self.item_model.encode(item_text, convert_to_tensor=True)

        user_output = self.user_fc(user_embedding)
        item_output = self.item_fc(item_embedding)
        # Log embeddings
        # self.user_emb = user_embedding
        # self.item_emb = item_embedding
        # print("User Embedding:", user_embedding)
        # print("Item Embedding:", item_embedding)
        # print("User Output:", user_output.size())
        # print("Item Output:", item_output.size())
        # Compute the dot product for each user-item pair in the batch
        dot_product = torch.sum(user_output * item_output, dim=1)
        # print("Dot Product:", dot_product)

        return dot_product

    def training_step(self, batch, batch_idx):
        user_texts = batch['user']
        item_texts = batch['item']
        ratings = batch['rating'].float()

        preds = self(user_texts, item_texts, 'train')
        preds = 4 * torch.sigmoid(preds) + 1
        # print(user_texts, item_texts, preds, ratings)
        loss = self.criterion(preds, ratings)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        user_texts = batch['user']
        item_texts = batch['item']
        ratings = batch['rating'].float()
        print(item_texts)
        preds = self(user_texts, item_texts, 'val')
        preds = 4 * torch.sigmoid(preds) + 1

        loss = self.criterion(preds, ratings)
        self.log('val_loss', loss)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=1e-5)

class PrintLossesCallback(Callback):
    def on_train_epoch_end(self, trainer, pl_module):
        train_loss = trainer.callback_metrics.get('train_loss')
        if train_loss is not None:
            pl_module.epoch_losses['train_loss'].append(train_loss.item())
            print(f"Epoch {trainer.current_epoch + 1}: Train Loss: {train_loss.item()}")

    def on_validation_epoch_end(self, trainer, pl_module):
        val_loss = trainer.callback_metrics.get('val_loss')
        if val_loss is not None:
            pl_module.epoch_losses['val_loss'].append(val_loss.item())
            print(f"Epoch {trainer.current_epoch + 1}: Val Loss: {val_loss.item()}")

# Example usage
# users = ["I love action movies", "Comedy movies are my favorite"]
# items = ["An action-packed thriller", "A hilarious comedy"]
# ratings = torch.tensor([5.0, 4.0])

train_loader = DataLoader(train_dataset, batch_size=2, drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=2, drop_last=True)  # For demonstration purposes, using the same data for validation

model = TwoTowerModel(user_model_name='paraphrase-MiniLM-L6-v2', item_model_name='paraphrase-MiniLM-L6-v2')

trainer = pl.Trainer(max_epochs=5, log_every_n_steps=1, callbacks=[PrintLossesCallback()], enable_progress_bar=False)
trainer.fit(model, train_loader, val_loader)

# Print losses after training completes
print("Epoch losses:")
for epoch in range(trainer.max_epochs):
    train_loss = model.epoch_losses['train_loss'][epoch] if epoch < len(model.epoch_losses['train_loss']) else 'N/A'
    val_loss = model.epoch_losses['val_loss'][epoch] if epoch < len(model.epoch_losses['val_loss']) else 'N/A'
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss}, Val Loss: {val_loss}")


D:\Anaconda\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type                | Params | Mode 
-----------------------------------------------------------
0 | user_model | SentenceTransformer | 22.7 M | train
1 | item_model | Sentenc

tensor([ 906, 3639], device='cuda:0')


D:\Anaconda\lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
D:\Anaconda\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


tensor([1977, 2011], device='cuda:0')
Epoch 1: Val Loss: 0.9361706972122192
tensor([ 906, 3639], device='cuda:0')
tensor([1977, 2011], device='cuda:0')
tensor([1513, 2424], device='cuda:0')
tensor([2302, 1404], device='cuda:0')
tensor([1266,  529], device='cuda:0')
Epoch 1: Val Loss: 2.4512267112731934
Epoch 1: Train Loss: 0.7173184156417847
tensor([ 906, 3639], device='cuda:0')
tensor([1977, 2011], device='cuda:0')
tensor([1513, 2424], device='cuda:0')
tensor([2302, 1404], device='cuda:0')
tensor([1266,  529], device='cuda:0')
Epoch 2: Val Loss: 2.3600029945373535
Epoch 2: Train Loss: 0.3913646638393402
tensor([ 906, 3639], device='cuda:0')
tensor([1977, 2011], device='cuda:0')
tensor([1513, 2424], device='cuda:0')
tensor([2302, 1404], device='cuda:0')
tensor([1266,  529], device='cuda:0')
Epoch 3: Val Loss: 2.317620038986206
Epoch 3: Train Loss: 0.21483533084392548
tensor([ 906, 3639], device='cuda:0')
tensor([1977, 2011], device='cuda:0')
tensor([1513, 2424], device='cuda:0')
tensor

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch losses:
Epoch 1: Train Loss: 0.7173184156417847, Val Loss: 0.9361706972122192
Epoch 2: Train Loss: 0.3913646638393402, Val Loss: 2.4512267112731934
Epoch 3: Train Loss: 0.21483533084392548, Val Loss: 2.3600029945373535
Epoch 4: Train Loss: 0.13959920406341553, Val Loss: 2.317620038986206
Epoch 5: Train Loss: 0.10530303418636322, Val Loss: 2.299961566925049


In [45]:
model.epoch_losses

{'train_loss': [1.1349900960922241,
  1.1209874153137207,
  1.1229170560836792,
  1.1273095607757568,
  1.1311087608337402,
  1.1337617635726929,
  1.1353812217712402,
  1.1362100839614868,
  1.1364727020263672,
  1.1363470554351807],
 'val_loss': [1.5670127868652344,
  1.1985464096069336,
  1.1731033325195312,
  1.16090726852417,
  1.1534429788589478,
  1.1482261419296265,
  1.144281268119812,
  1.1411384344100952,
  1.1385533809661865,
  1.13637375831604,
  1.1345018148422241]}

In [ ]:
{1579: 2028    1.934932
589     1.828363
2762    1.772296
527     1.746241
858     1.700419
dtype: float64, 5627: 260     2.251319
608     1.876114
1270    1.820516
480     1.663024
1097    1.595977}

In [47]:
def get_top_n_recommendations(user_text, model, item_dataset, n=5):
    item_texts = [item_dataset[i] for i in range(len(item_dataset))]
    item_texts= torch.tensor(item_texts)
    preds = model.forward(user_text, item_texts, 'test')
    preds = 4 * torch.sigmoid(preds) + 1
    top_n_indices = torch.argsort(preds, descending=True)[:n]
    top_n_items = [item_dataset[i] for i in top_n_indices]
    top_n_scores = preds[top_n_indices]

    top_n_series = pd.Series(data=top_n_scores.detach().numpy(), index=top_n_items)
    return {top_n_series}


In [50]:
def get_top_n_recommendations(user_texts, user_ids, model, item_dataset, n=5):
    recommendations = {}

    for user_text, user_id in zip(user_texts, user_ids):
        item_texts = [item_dataset[i] for i in range(len(item_dataset))]
        item_texts = torch.tensor(item_texts)
        preds = model.forward(user_text, item_texts, 'test')
        preds = 4 * torch.sigmoid(preds) + 1
        top_n_indices = torch.argsort(preds, descending=True)[:n]
        top_n_items = [item_dataset[i] for i in top_n_indices]
        top_n_scores = preds[top_n_indices]

        top_n_series = pd.Series(data=top_n_scores.detach().numpy(), index=top_n_items)
        recommendations[user_id] = top_n_series

    return recommendations

In [52]:
# Example user text
# user_text = "profession: doctor [SEP] age range: 65-74 [SEP] gender: Male"
user_text = "profession: scientist [SEP] age range: 75+ [SEP] gender: Male"

top_recommendations = get_top_n_recommendations(user_text, [1], model, random_item_ids, n=5)
# top_recommendations = get_top_n_recommendations(user_text, model, movies, n=5)
top_recommendations

{1: 2347    2.318550
 364     2.222605
 1327    2.192921
 2005    2.169346
 3408    1.947947
 dtype: float32}

In [49]:
top_recommendations

2347    4.406095
364     4.216607
2005    4.097148
1327    3.700044
3379    3.631617
dtype: float32

In [13]:
t_item_texts = [movie_features_dict[movieId] for movieId in random_item_ids]
t_item_texts

['title: Against All Odds (1984) [SEP] genres: Romance',
 "title: What's Eating Gilbert Grape (1993) [SEP] genres: Drama",
 'title: Pope of Greenwich Village, The (1984) [SEP] genres: Action',
 "title: Goonies, The (1985) [SEP] genres: Adventure|Children's|Fantasy",
 'title: Amityville Horror, The (1979) [SEP] genres: Horror',
 'title: Powder (1995) [SEP] genres: Drama|Sci-Fi',
 "title: Lion King, The (1994) [SEP] genres: Animation|Children's|Musical",
 'title: Erin Brockovich (2000) [SEP] genres: Drama',
 'title: On the Beach (1959) [SEP] genres: Drama',
 'title: Mona Lisa (1986) [SEP] genres: Comedy|Thriller']

In [14]:
# data = {
#     "User": users,  # Repeating users to match the number of ratings
#     "Movie": movies,
#     "Rating": ratings
# }
data = {
    "User": random_user_texts,  # Repeating users to match the number of ratings
    "Movie": [f"{item_id} {text}" for item_id, text in zip(random_item_ids, t_item_texts)],
    "Rating": random_ratings
}


In [15]:
df = pd.DataFrame(data)


In [16]:
df

,User,Movie,Rating
0,occupation: executive/managerial [SEP] age: 25...,3206 title: Against All Odds (1984) [SEP] genr...,2
1,occupation: artist [SEP] age: 18-24 [SEP] gend...,337 title: What's Eating Gilbert Grape (1993) ...,3
2,occupation: college/grad student [SEP] age: 18...,"2347 title: Pope of Greenwich Village, The (19...",5
3,occupation: customer service [SEP] age: 45-49 ...,"2005 title: Goonies, The (1985) [SEP] genres: ...",3
4,occupation: academic/educator [SEP] age: 35-44...,"1327 title: Amityville Horror, The (1979) [SEP...",3
5,occupation: self-employed [SEP] age: 25-34 [SE...,24 title: Powder (1995) [SEP] genres: Drama|Sc...,3
6,occupation: other or not specified [SEP] age: ...,"364 title: Lion King, The (1994) [SEP] genres:...",4
7,occupation: college/grad student [SEP] age: 25...,3408 title: Erin Brockovich (2000) [SEP] genre...,5
8,occupation: writer [SEP] age: 50-55 [SEP] gend...,3379 title: On the Beach (1959) [SEP] genres: ...,4
9,occupation: academic/educator [SEP] age: 35-44...,2349 title: Mona Lisa (1986) [SEP] genres: Com...,3


In [17]:
user_text = "profession: scientist [SEP] age range: 75+ [SEP] gender: Male"

In [18]:
top_recommendations

[(2347, tensor(4.4061, grad_fn=<UnbindBackward0>)),
 (364, tensor(4.2166, grad_fn=<UnbindBackward0>)),
 (2005, tensor(4.0971, grad_fn=<UnbindBackward0>)),
 (1327, tensor(3.7000, grad_fn=<UnbindBackward0>)),
 (3379, tensor(3.6316, grad_fn=<UnbindBackward0>)),
 (24, tensor(3.5800, grad_fn=<UnbindBackward0>)),
 (3206, tensor(3.3498, grad_fn=<UnbindBackward0>)),
 (337, tensor(3.2846, grad_fn=<UnbindBackward0>)),
 (2349, tensor(3.1287, grad_fn=<UnbindBackward0>)),
 (3408, tensor(3.1109, grad_fn=<UnbindBackward0>))]

In [53]:
from evaluator import EvaluateMetrics
n=5
# Evaluate the recommendations
evaluator = EvaluateMetrics(test_ratings)
evaluation_results = evaluator.evaluate_recommendations(top_recommendations, k=n)

print(f"NDCG@{n}: {evaluation_results['NDCG@k']:.4f}")
print(f"MRR@{n}: {evaluation_results['MRR@k']:.4f}")
print(f"HR@{n}: {evaluation_results['HR@k']:.4f}")

NDCG@5: 0.3869
MRR@5: 0.2000
HR@5: 1.0000


In [104]:
model.user_emb

tensor([[ 8.3276e-01,  8.2051e-02,  3.0309e-01,  2.1005e-01, -3.0842e-01,
          8.1448e-01, -9.2465e-02, -3.7289e-01,  8.7239e-02,  6.1374e-01,
         -1.7548e-01, -2.6898e-01,  1.0095e-01,  2.0776e-02, -2.9030e-02,
          3.1094e-01, -1.5206e-01, -1.8223e-01,  7.2254e-01,  2.6763e-01,
         -1.7961e-01,  2.1395e-01,  2.5993e-01, -6.7496e-01,  5.0078e-01,
         -6.1833e-01, -2.8589e-01,  1.1161e-01, -2.1280e-01, -2.1083e-01,
          3.3463e-01,  4.1669e-01,  4.4386e-01, -1.3550e-02,  5.8307e-02,
         -1.3067e-01, -1.0299e+00,  1.5860e-01, -2.2157e-02,  3.0831e-01,
         -4.3915e-02, -7.7502e-02, -1.4880e-01, -6.4330e-01, -1.1070e-02,
          1.3839e-01, -1.5811e-01, -4.4167e-04,  1.1641e-01,  3.9993e-01,
         -9.0280e-01, -6.9036e-01,  1.4046e-01,  3.3495e-01, -3.1607e-01,
         -4.2657e-01,  7.9768e-03,  2.0949e-02,  2.9584e-01,  6.0345e-02,
         -2.0942e-01,  3.5031e-01, -7.5630e-01, -7.8596e-02,  1.1698e-02,
          1.8370e-01,  2.7463e-01, -2.

In [105]:
model.item_emb

tensor([[ 3.8231e-02, -5.1786e-01,  2.8932e-01, -6.5171e-02, -2.2142e-01,
          4.8638e-01, -3.7634e-01, -1.0395e-03,  5.1911e-02,  2.2404e-01,
          1.9158e-01, -1.5225e-01, -6.5987e-01,  3.5734e-02,  1.8200e-01,
         -1.5093e-01,  4.0238e-01,  6.6269e-02,  6.9296e-01,  5.6623e-01,
          1.0681e-01,  6.3442e-01,  2.1215e-01, -6.2358e-01,  1.9043e-01,
         -5.2030e-01,  1.6891e-01,  3.1781e-01, -6.8459e-01, -1.2312e-01,
          4.7328e-01,  4.6238e-02, -2.1466e-01,  5.2682e-02,  1.6736e-01,
          1.9360e-01,  3.4173e-01,  2.5968e-01,  2.0081e-01, -5.5312e-01,
         -3.8487e-01,  2.2561e-01,  1.7053e-01,  3.6520e-01, -2.2282e-01,
         -1.8844e-01, -3.3730e-01, -5.5030e-01,  5.9279e-01,  2.9978e-01,
         -6.6722e-01, -1.8076e-01, -3.6294e-01,  1.3673e-01,  6.3215e-01,
         -4.3464e-02,  1.1103e-01,  3.4903e-01,  9.4543e-02,  1.7410e-01,
          2.7457e-01, -1.4252e-01, -6.0077e-01,  1.5249e-01,  1.1750e-01,
         -4.0101e-01,  1.4326e-02,  4.

In [107]:
# Calculate the dot product
dot_product = torch.sum(model.user_emb * model.item_emb, dim=1)

print("Dot product:", dot_product.item())

Dot product: 16.750228881835938
